#### Logging function runs

In [97]:
from functools import wraps

In [98]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename=f'{orig_func.__name__}.log', level=logging.INFO, force=True)

    # @wraps(orig_func)
    def wrapper_log(*args, **kwargs):
        logging.info(
            f'Function {orig_func.__name__} (wrapped in wrapper_log) ran with args: {args}, and kwargs: {kwargs}')
        print(f'Function {orig_func.__name__} (wrapped in wrapper_log) ran with args: {args}, and kwargs: {kwargs}')
        return orig_func(*args, **kwargs)

    return wrapper_log

In [99]:
# This decorator (my_logger) pre-configured wrapper to remember
# display_info as original function, and
# initiated logging

# The pre-configured wrapper waits to be executed
@my_logger
def display_info(name, age):
    print(f'display_info ran with arguments ({name}, {age})')

In [100]:
# This executed wrapper with argument
display_info('Beth', 25)

Function display_info (wrapped in wrapper_log) ran with args: ('Beth', 25), and kwargs: {}
display_info ran with arguments (Beth, 25)


In [101]:
# We can check things are logged (additional functionality added via decorator)
!cat display_info.log

INFO:root:Function display_info (wrapped in wrapper_log) ran with args: ('Beth', 25), and kwargs: {}


#### Timing example

In [102]:
def my_timer(orig_func):
    import time

    # @wraps(orig_func)
    def wrapper_time(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print(f'Function {orig_func.__name__} (wrapped in wrapper_time) ran in: {t2:.8f} sec')
        return result

    return wrapper_time

In [103]:
@my_timer
def display_info(name, age):
    print(f'display_info ran with arguments ({name}, {age})')

In [104]:
display_info('Tom', 25)

display_info ran with arguments (Tom, 25)
Function display_info (wrapped in wrapper_time) ran in: 0.00013947 sec


#### Stacked decorators

In [105]:
@my_timer # receive wrapper_log as argument, return wrapper_time function (timing and execute wrapper_log) assigned to display_info
@my_logger # receive display_info as argument, initiate display_info.log, return wrapper_log function (logging and execute display_info)
def display_info(name, age):
    print(f'display_info ran with arguments ({name}, {age})')

# Once executed, it runs wrapper_time 1) time wrapper_log start 2) runs wrapper_log: logging display_info, display_info) 3) print wrapper_log time 4) return execution of wrapper_log (None)

What the above did is that it first do my_logger decorator

This makes logging correct

However, for my_timer, it is

    display_info = my_timer(my_logger(display_info))

and `my_logger(display_info).__name__` is `wrapper_log`

In [106]:
display_info('Tom', 20)

Function display_info (wrapped in wrapper_log) ran with args: ('Tom', 20), and kwargs: {}
display_info ran with arguments (Tom, 20)
Function wrapper_log (wrapped in wrapper_time) ran in: 0.00049424 sec


If we change the order, then obviously the wrapper_time will be the function named passed to my_logger, so there will be no display_info.log, but wrapper_time.log

In [107]:
@my_logger # receive wrapper_time as argument, initiate wrapper_time.log, return wrapper_log (log wrapper_time, execute wrapper_time) function assigned to display_info
@my_timer # receive display_info as argument, return wrapper_time function (time start, display_info, print time)
def display_info(name, age):
    print(f'display_info ran with arguments ({name}, {age})')

# Once executed, it runs wrapper_log 1) log wrapper_time 2) run wrapper_time: a) time display_info start b) run display_info c) print display_info time d) return execution of display_info (None)

In [108]:
display_info('Max', 35)

Function wrapper_time (wrapped in wrapper_log) ran with args: ('Max', 35), and kwargs: {}
display_info ran with arguments (Max, 35)
Function display_info (wrapped in wrapper_time) ran in: 0.00000691 sec
